[Stable Diffusion XL Base 1.0](https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lwieske/stable-diffusion-xl-demos/blob/main/stable_diffusion_xl_demos_colab.ipynb)

In [23]:
import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

!rm -rf "/content/drive/MyDrive/README.md"
!rm -rf "/content/drive/MyDrive/images"

!mkdir -p "/content/drive/MyDrive/images"

Mounted at /content/drive


In [24]:
!pip install --quiet --upgrade transformers accelerate diffusers invisible_watermark mediapy

In [25]:
!nvidia-smi

Sat Sep 16 05:38:00 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   75C    P0    33W /  70W |  12951MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [26]:
import random
import sys

import torch

import mediapy as media

from diffusers import DiffusionPipeline

from tqdm.notebook import tqdm

In [27]:
use_refiner = False

In [28]:
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [29]:
if use_refiner:
  refiner = DiffusionPipeline.from_pretrained(
      "stabilityai/stable-diffusion-xl-refiner-1.0",
      text_encoder_2=pipe.text_encoder_2,
      vae=pipe.vae,
      torch_dtype=torch.float16,
      use_safetensors=True,
      variant="fp16",
  )

  refiner = refiner.to("cuda")

  pipe.enable_model_cpu_offload()
else:
  pipe = pipe.to("cuda")

In [30]:
prompts = [
  "franoise nielly",
  "crowded beach scene",
  "market scene from a vendors perspective emphasizing colorful stalls and busy shoppers and an array of products",
  "draw data center in charcoal style",
  "paint data center in the style of van gogh",
  "Donald Trump in prison behind bars",
  "modern building with organic design in Hadid style",
  "Pope Francis mixing live as DJ in a nightclub",
  "fantasy medieval village world inside a glass sphere",
  "romantic painting of a ship sailing in a stormy sea with dramatic lighting and powerful waves",
  "black and white street photography of a rainy night in New York and reflections on wet pavement",
  "documentary style photography of a bustling marketplace with spices and textiles",
  "isometric digital art of a medieval village with thatched roofs and market square and townsfolk",
  "minimalist digital artwork with an abstract geometric pattern and a harmonious color palette",
  "King Arthur Round Table with Bengio and Hinton and LeCun",
  "abstract painting representing the sound of jazz music using vibrant colors and erratic shapes",
  "abstract painting representing the sound of rock music using vibrant colors and erratic shapes",
  "digital art with straight houses in foreground and background digital art with sharp focus in anime style",
  "gingerbread house diorama in focus with crunch cereal",
  "cartoon cow portrait digital art in high definition and Andy Warhol style",
  "dystopian image with earth in chain",
  "earth in intensive care unit",
]

In [31]:
imgdescs = []

for prompt in prompts:
  seed = random.randint(0, sys.maxsize)

  images = pipe(
    prompt = prompt,
    output_type = "latent" if use_refiner else "pil",
    generator = torch.Generator("cuda").manual_seed(seed),
  ).images

  if use_refiner:
    images = refiner(
      prompt = prompt,
      image = images,
    ).images

  file = prompt.replace(' ', '_')

  print(f"Prompt:\t{prompt}\nSeed:\t{seed}\nFile:\timages/{file}.png\n")

  imgdescs.append( {'Prompt': prompt, 'Seed': seed, 'File': file} )

  media.write_image('/content/drive/MyDrive/images/' + file + '.png', images[0])

  0%|          | 0/50 [00:00<?, ?it/s]

Prompt:	franoise nielly
Seed:	6013084719118494839
File:	images/franoise_nielly.png



  0%|          | 0/50 [00:00<?, ?it/s]

Prompt:	crowded beach scene
Seed:	2922738400769003415
File:	images/crowded_beach_scene.png



  0%|          | 0/50 [00:00<?, ?it/s]

...

Prompt:	cartoon cow portrait digital art in high definition and Andy Warhol style
Seed:	2931055462342485031
File:	images/cartoon_cow_portrait_digital_art_in_high_definition_and_Andy_Warhol_style.png



  0%|          | 0/50 [00:00<?, ?it/s]

Prompt:	dystopian image with earth in chain
Seed:	8402555520972316634
File:	images/dystopian_image_with_earth_in_chain.png



  0%|          | 0/50 [00:00<?, ?it/s]

Prompt:	earth in intensive care unit
Seed:	503752777397095896
File:	images/earth_in_intensive_care_unit.png



In [33]:
for imgdesc in imgdescs:
  prompt = imgdesc['Prompt']
  seed = imgdesc['Seed']
  file = imgdesc['File']

  print(f"Prompt:\t{prompt}\nSeed:\t{seed}",          file=open('/content/drive/MyDrive/README.md', 'a'))
  print(f"![images/{file}.png](images/{file}.png)\n", file=open('/content/drive/MyDrive/README.md', 'a'))
